This was run on Google Colab with a GPU. It shows a more state-of-the-art approach to this problem.

In [5]:
import torch

In [6]:
torch.cuda.is_available()

True

In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tweets.csv to tweets.csv
User uploaded file "tweets.csv" with length 7823952 bytes


In [8]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.7 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 636 kB 45.6 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
import math

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer

In [10]:
tweets_df = pd.read_csv("tweets.csv")


In [11]:
tweets_df

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655
...,...,...,...,...,...,...,...,...,...,...
52537,ddlovato,Life couldn't be better right now. 😊,NaN,06/01/2015 23:10,5.526030e+17,en,NaN,NaN,32799,23796
52538,ddlovato,First Monday back in action. I'd say 21.6 mile...,NaN,06/01/2015 02:17,5.522880e+17,en,NaN,NaN,21709,12511
52539,ddlovato,"Crime shows, buddy, snuggles = the perfect Sun...",NaN,05/01/2015 03:42,5.519470e+17,en,NaN,NaN,25269,15583
52540,ddlovato,❄️ http://t.co/sHCFdPpGPa,NaN,05/01/2015 00:06,5.518920e+17,und,NaN,NaN,15985,10456


In [12]:
unique_people = tweets_df['author'].unique()
print(unique_people)
NUM_CLASSES = len(unique_people)

# assign each person a number
id_to_person = {i: unique_people[i] for i in range(len(unique_people))}
person_to_id = {v:k for k,v in id_to_person.items()}

# create a column of author ids
tweets_df['author_id'] = tweets_df['author'].apply(lambda x: person_to_id[x])

['katyperry' 'justinbieber' 'taylorswift13' 'BarackObama' 'rihanna'
 'YouTube' 'ladygaga' 'TheEllenShow' 'Twitter' 'jtimberlake'
 'KimKardashian' 'britneyspears' 'Cristiano' 'selenagomez' 'cnnbrk'
 'jimmyfallon' 'ArianaGrande' 'shakira' 'instagram' 'ddlovato']


In [13]:
!pip install simpletransformers
!pip install tensorboardx

     |████████████████████████████████| 230 kB 5.4 MB/s 
     |████████████████████████████████| 1.7 MB 35.5 MB/s 
     |████████████████████████████████| 8.3 MB 39.9 MB/s 
     |████████████████████████████████| 270 kB 24.3 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 41.6 MB/s 
     |████████████████████████████████| 180 kB 40.6 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 139 kB 26.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 1.3 MB 42.7 MB/s 
     |████████████████████████████████| 243 kB 43.2 MB/s 
     |████████████████████████████████| 123 kB 42.9 MB/s 
     |████████████████████████████████| 294 kB 35.6 MB/s 
     |████████████████████████████████| 142 kB 48.9 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 4.3 MB 36.4 MB/s 
     |█████████████████

     |████████████████████████████████| 124 kB 5.5 MB/s eta 0:00:01


In [14]:
from simpletransformers.classification import ClassificationModel


In [27]:
# We only do 2 epochs. More will result in a better accuracy.
model = ClassificationModel('roberta', 'roberta-base', use_cuda=True, num_labels=NUM_CLASSES, args={
    'train_batch_size': 64,
    'num_train_epochs': 2,
    'max_seq_length': 128,
    'learning_rate': 2e-5,
})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [22]:

np.random.seed(7)

# first column should be tweet, second column should be label
df_full = tweets_df[['content', 'author_id']].sample(frac=1.0)

n_train = 10000
n_test = 10000

df_train = df_full[:n_train]
df_test = df_full[n_train:n_train+n_test]


In [47]:
model.train_model(df_train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/157 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:922: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


(157, 1.7726426823124004)

In [48]:
raw_outputs_train = model.eval_model(df_train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/10000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1250 [00:00<?, ?it/s]

In [49]:
# train acc
(np.argmax(raw_outputs_train[1], axis=1) == df_train['author_id'].values).mean()

0.6135

In [50]:
raw_outputs_test = model.eval_model(df_test)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/10000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1250 [00:00<?, ?it/s]

In [51]:
# test acc
(np.argmax(raw_outputs_test[1], axis=1) == df_test['author_id'].values).mean()

0.5797